<a href="https://colab.research.google.com/github/ArthurTerozendi/classificador-pefil-voto-candidato/blob/main/classificador_pefil_voto_candidato.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Clonando repositório

In [2]:
!git clone https://github.com/ArthurTerozendi/classificador-pefil-voto-candidato.git

Cloning into 'classificador-pefil-voto-candidato'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 1), reused 5 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), done.


# Imports

In [3]:
import pandas as pd

In [95]:
bem_candidatos = pd.read_csv('/content/classificador-pefil-voto-candidato/database/bem_candidato_2022_MG.csv', ',', encoding='latin1', usecols=['SQ_CANDIDATO', 'VR_BEM_CANDIDATO'])
consulta_cand = pd.read_csv('/content/classificador-pefil-voto-candidato/database/consulta_cand_2022_MG.csv', ',', encoding='latin1', usecols=['DS_CARGO', 'SQ_CANDIDATO', 'NM_CANDIDATO', 'DT_NASCIMENTO', 'DS_GENERO', 'DS_ESTADO_CIVIL', 'DS_COR_RACA', 'DS_OCUPACAO', 'ST_REELEICAO'])
consulta_cand

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


,DS_CARGO,SQ_CANDIDATO,NM_CANDIDATO,DT_NASCIMENTO,DS_GENERO,DS_ESTADO_CIVIL,DS_COR_RACA,DS_OCUPACAO,ST_REELEICAO
0,DEPUTADO FEDERAL,130001595987,JULIANA APARECIDA DE SOUZA GUERRA,23/11/1984,FEMININO,DIVORCIADO(A),BRANCA,CABELEIREIRO E BARBEIRO,N
1,DEPUTADO FEDERAL,130001613363,Sï¿½RGIO CAMPOS SOUZA,29/07/1971,MASCULINO,DIVORCIADO(A),PRETA,EMPRESï¿½RIO,N
2,DEPUTADO FEDERAL,130001610915,EDMILSON DE OLIVEIRA PEREIRA,18/03/1981,MASCULINO,CASADO(A),BRANCA,REPRESENTANTE COMERCIAL,N
3,DEPUTADO ESTADUAL,130001607719,JARDEL MOIA MONTEIRO,24/01/1984,MASCULINO,CASADO(A),PARDA,OUTROS,N
4,DEPUTADO FEDERAL,130001668896,APARECIDA MOREIRA DE OLIVEIRA PAIVA,22/09/1964,FEMININO,CASADO(A),BRANCA,ADVOGADO,N
...,...,...,...,...,...,...,...,...,...
2557,SENADOR,130001653898,IRANI DA SILVA GOMES,11/05/1974,MASCULINO,CASADO(A),BRANCA,OUTROS,N
2558,DEPUTADO ESTADUAL,130001634285,HENRIQUE BARBOSA DE OLIVEIRA,07/08/1978,MASCULINO,CASADO(A),BRANCA,EMPRESï¿½RIO,N
2559,DEPUTADO ESTADUAL,130001615227,ADEMILDE FRANCISCO DIAS,30/12/1969,FEMININO,SOLTEIRO(A),PARDA,COMERCIANTE,N
2560,DEPUTADO ESTADUAL,130001615280,ISABELLA GONï¿½ALVES MIRANDA,20/11/1988,FEMININO,DIVORCIADO(A),BRANCA,VEREADOR,N


- Somatório do valor dos bens de cada canditado

In [96]:
bem_candidatos['VR_BEM_CANDIDATO'] = bem_candidatos['VR_BEM_CANDIDATO'].apply(lambda x: x.replace(',', '.'))
bem_candidatos['VR_BEM_CANDIDATO'] = pd.to_numeric(bem_candidatos['VR_BEM_CANDIDATO'], downcast='float')
bem_candidatos_sum = bem_candidatos.groupby(['SQ_CANDIDATO']).sum()
bem_candidatos_sum

,VR_BEM_CANDIDATO
SQ_CANDIDATO,
130001595976,2.195987e+07
130001595978,5.700000e+05
130001595979,3.332248e+05
130001595983,9.090000e+05
130001595984,2.073415e+06
...,...
130001737057,5.000000e+05
130001737059,1.500000e+04
130001737060,1.360000e+04


- Pegando apenas os Deputados

In [97]:
dEstadual = consulta_cand.query("DS_CARGO == 'DEPUTADO ESTADUAL'", inplace=False)
dFederal = consulta_cand.query("DS_CARGO == 'DEPUTADO FEDERAL'", inplace=False)
deputados = pd.concat([dEstadual, dFederal])
deputados

,DS_CARGO,SQ_CANDIDATO,NM_CANDIDATO,DT_NASCIMENTO,DS_GENERO,DS_ESTADO_CIVIL,DS_COR_RACA,DS_OCUPACAO,ST_REELEICAO
3,DEPUTADO ESTADUAL,130001607719,JARDEL MOIA MONTEIRO,24/01/1984,MASCULINO,CASADO(A),PARDA,OUTROS,N
5,DEPUTADO ESTADUAL,130001615626,REINALDO LUCIO DA SILVA,30/08/1972,MASCULINO,CASADO(A),PARDA,OUTROS,N
6,DEPUTADO ESTADUAL,130001634278,ELIES GOMES COSTA,23/02/1997,MASCULINO,CASADO(A),PARDA,EMPRESï¿½RIO,N
9,DEPUTADO ESTADUAL,130001669318,JOSE GONZAGA DE SOUZA,19/05/1961,MASCULINO,CASADO(A),PRETA,LOCUTOR E COMENTARISTA DE Rï¿½DIO E TELEVISï¿½...,N
13,DEPUTADO ESTADUAL,130001607418,SILVIA DA CRUZ MESSIAS,13/08/1964,FEMININO,CASADO(A),BRANCA,VEREADOR,N
...,...,...,...,...,...,...,...,...,...
2549,DEPUTADO FEDERAL,130001607279,DINEY LENON DE PAULO,16/06/1981,MASCULINO,SOLTEIRO(A),BRANCA,PROFESSOR DE ENSINO Mï¿½DIO,N
2551,DEPUTADO FEDERAL,130001606187,ANGELITA MOREIRA ROCHA,20/09/1966,FEMININO,SOLTEIRO(A),BRANCA,APOSENTADO (EXCETO SERVIDOR Pï¿½BLICO),N
2552,DEPUTADO FEDERAL,130001606191,ERICK SANTOS NAZARETH,20/12/1980,MASCULINO,CASADO(A),PARDA,EMPRESï¿½RIO,N
2553,DEPUTADO FEDERAL,130001607565,RICARDO GONï¿½ALVES CAMPOS,13/11/1970,MASCULINO,CASADO(A),BRANCA,ENGENHEIRO,N


- Junção das duas tabelas

In [100]:
registroCand = pd.merge(deputados, bem_candidatos_sum, how='left', left_on='SQ_CANDIDATO', right_on='SQ_CANDIDATO')
registroCand

,DS_CARGO,SQ_CANDIDATO,NM_CANDIDATO,DT_NASCIMENTO,DS_GENERO,DS_ESTADO_CIVIL,DS_COR_RACA,DS_OCUPACAO,ST_REELEICAO,VR_BEM_CANDIDATO
0,DEPUTADO ESTADUAL,130001607719,JARDEL MOIA MONTEIRO,24/01/1984,MASCULINO,CASADO(A),PARDA,OUTROS,N,3.490000e+04
1,DEPUTADO ESTADUAL,130001615626,REINALDO LUCIO DA SILVA,30/08/1972,MASCULINO,CASADO(A),PARDA,OUTROS,N,2.200000e+04
2,DEPUTADO ESTADUAL,130001634278,ELIES GOMES COSTA,23/02/1997,MASCULINO,CASADO(A),PARDA,EMPRESï¿½RIO,N,6.800000e+05
3,DEPUTADO ESTADUAL,130001669318,JOSE GONZAGA DE SOUZA,19/05/1961,MASCULINO,CASADO(A),PRETA,LOCUTOR E COMENTARISTA DE Rï¿½DIO E TELEVISï¿½...,N,NaN
4,DEPUTADO ESTADUAL,130001607418,SILVIA DA CRUZ MESSIAS,13/08/1964,FEMININO,CASADO(A),BRANCA,VEREADOR,N,1.592013e+05
...,...,...,...,...,...,...,...,...,...,...
2509,DEPUTADO FEDERAL,130001607279,DINEY LENON DE PAULO,16/06/1981,MASCULINO,SOLTEIRO(A),BRANCA,PROFESSOR DE ENSINO Mï¿½DIO,N,NaN
2510,DEPUTADO FEDERAL,130001606187,ANGELITA MOREIRA ROCHA,20/09/1966,FEMININO,SOLTEIRO(A),BRANCA,APOSENTADO (EXCETO SERVIDOR Pï¿½BLICO),N,NaN
2511,DEPUTADO FEDERAL,130001606191,ERICK SANTOS NAZARETH,20/12/1980,MASCULINO,CASADO(A),PARDA,EMPRESï¿½RIO,N,4.500000e+04
2512,DEPUTADO FEDERAL,130001607565,RICARDO GONï¿½ALVES CAMPOS,13/11/1970,MASCULINO,CASADO(A),BRANCA,ENGENHEIRO,N,2.537668e+06
